<a href="https://colab.research.google.com/github/marcosfelt/interview_study_plan/blob/main/algorithms/dynamic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Programming

From this youtube course:  https://www.youtube.com/watch?v=oBt53YbR9Kk

## Fibonacci

The fibonacci series is a positive integer series where each element past the second is the sum of the previous two:

1, 1, 2, 3, 5, 8, 13, ...

Can we write an algorithm to caclulate the series for any $n$?

In [3]:
# Slow implementation
def fib(n):
  if n<=2:
    return 1
  else:
    return fib(n-1) + fib(n-2)

For small $n$ the above implementation is fine:

In [4]:
print("7:", fib(7))
print("8:", fib(8))
print("13:", fib(13))

7: 13
8: 21
13: 233


However, for slightly larger n, it will stall (try running below):

In [ ]:
print("50:", fib(50))

We can think of the call stack as below:

![](https://github.com/marcosfelt/interview_study_plan/blob/main/static/fibonacci_tree.png?raw=true)

Since each of the subtrees is repeated, we can _memoize_ or store the result of the calculation the first time we do it. Therefore, we transform the algorithm from $O(2^n)$ to $O(n)$.

In [13]:
# Fast implementation
def fib_memo(n, memo=None):
  if memo is None:
    memo = {}
  if n<=2:
    return 1
  elif n in memo:
    return memo[n]
  else:
    r = fib_memo(n-1, memo) + fib_memo(n-2, memo)
    memo[n] = r
    return r

In [14]:
print("50:", fib_memo(50))

50: 12586269025


Yay! We can now calculate the fibonacci series of large numbers in linear time.

## Grid Traveler

In [15]:
def grid_traveler(m,n):
  # Base case
  if m==1 and n ==1:
    return 1
  # Trivial case
  if m==0 or n==0:
    return 0
  # Down + # Right
  return grid_traveler(m-1, n) + grid_traveler(m, n-1)

Works for small $n$

In [20]:
print("1,1",grid_traveler(1,1))
print("2,3:",grid_traveler(2,3))


1,1 1
2,3: 3


Bigger $n$ is slower

In [ ]:
print("18,18:",grid_traveler(18,18))

We found in our analyssi that:
- We want to use memoization for subtrees
- We can leverage symmetry between rows and columns (since its down and right always). 

In [23]:
def grid_traveler_memo(m,n, memo=None):
  if memo is None:
    memo = {}

  # Check if (m,n) in memo
  if (m,n) in memo:
    return memo[(m,n)]
  elif (n,m) in memo:
    return memo[(n,m)]

  # Base case
  if m==1 and n ==1:
    return 1
  # Trivial case
  elif m==0 or n==0:
    return 0
  # Down + Right
  else:
    count = grid_traveler_memo(m-1, n, memo) + grid_traveler_memo(m, n-1, memo)
    memo[(m,n)] = count
    return count

In [24]:
print("18,18:",grid_traveler_memo(18,18))

18,18: 2333606220
